<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 3 - Roshan Lulu

### Regression and Classification with the Ames Housing Data

---

You have just joined a new "full stack" real estate company in Ames, Iowa. The strategy of the firm is two-fold:
- Own the entire process from the purchase of the land all the way to sale of the house, and anything in between.
- Use statistical analysis to optimize investment and maximize return.

The company is still small, and though investment is substantial the short-term goals of the company are more oriented towards purchasing existing houses and flipping them as opposed to constructing entirely new houses. That being said, the company has access to a large construction workforce operating at rock-bottom prices.

This project uses the [Ames housing data recently made available on kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques).

In [223]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1. Estimating the value of homes from fixed characteristics.

---

Your superiors have outlined this year's strategy for the company:
1. Develop an algorithm to reliably estimate the value of residential houses based on *fixed* characteristics.
2. Identify characteristics of houses that the company can cost-effectively change/renovate with their construction team.
3. Evaluate the mean dollar value of different renovations.

Then we can use that to buy houses that are likely to sell for more than the cost of the purchase plus renovations.

Your first job is to tackle #1. You have a dataset of housing sale data with a huge amount of features identifying different aspects of the house. The full description of the data features can be found in a separate file:

    housing.csv
    data_description.txt
    
You need to build a reliable estimator for the price of the house given characteristics of the house that cannot be renovated. Some examples include:
- The neighborhood
- Square feet
- Bedrooms, bathrooms
- Basement and garage space

and many more. 

Some examples of things that **ARE renovate-able:**
- Roof and exterior features
- "Quality" metrics, such as kitchen quality
- "Condition" metrics, such as condition of garage
- Heating and electrical components

and generally anything you deem can be modified without having to undergo major construction on the house.

---

**Your goals:**
1. Perform any cleaning, feature engineering, and EDA you deem necessary.
- Be sure to remove any houses that are not residential from the dataset.
- Identify **fixed** features that can predict price.
- Train a model on pre-2010 data and evaluate its performance on the 2010 houses.
- Characterize your model. How well does it perform? What are the best estimates of price?

> **Note:** The EDA and feature engineering component to this project is not trivial! Be sure to always think critically and creatively. Justify your actions! Use the data description file!

In [224]:
# Load the data
house = pd.read_csv('./housing.csv')

In [240]:
print('Original shape of DF:')
print(house.shape)

Original shape of DF:
(1450, 80)


In [226]:
# Set minimum number of rows and columns to be displayed as we are dealing with a large dataset
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [241]:
# A:
print('DF head:')
house.head(2)

DF head:


,Ext_Type_MSSubClass,Ext_Type_MSZoning,Ext_F_LotFrontage,Int_F_LotArea,Ext_C_Street,Ext_C_Alley,Int_F_LotShape,Int_F_LandContour,Int_C_Util_Utilities,Int_F_LotConfig,Ext_F_LandSlope,Ext_F_Neighborhood,Ext_F_Condition1,Ext_F_Condition2,Int_F_BldgType,Int_F_HouseStyle,Int_F_OverallQual,Int_C_OverallCond,Ext_F_Year_YearBuilt,Ext_F_Year_YearRemodAdd,Int_C_Mat_RoofStyle,Int_C_Mat_RoofMatl,Int_C_Mat_Exterior1st,Int_C_Mat_Exterior2nd,Int_C_Mat_MasVnrType,Int_C_Mat_Sqft_MasVnrArea,Int_C_Mat_Qual_ExterQual,Int_C_Mat_Cond_ExterCond,Int_F_Mat_Foundation,Int_F_Bsmnt_Qual_BsmtQual,Int_F_Bsmnt_Cond_BsmtCond,Int_C_Bsmnt_BsmtExposure,Int_F_Bsmnt_Qual_BsmtFinType1,Int_F_Bsmnt_Sqft_BsmtFinSF1,Int_F_Bsmnt_Qual_BsmtFinType2,Int_F_Bsmnt_Sqft_BsmtFinSF2,Int_F_Bsmnt_Sqft_BsmtUnfSF,Int_F_Bsmnt_Sqft_TotalBsmtSF,Int_C_Util_Heating,Int_C_Qual_Util_HeatingQC,Int_F_Util_CentralAir,Int_F_Util_Electrical,Int_F_Sqft_1stFlrSF,Int_F_Sqft_2ndFlrSF,Int_C_Sqft_LowQualFinSF,Int_F_Sqft_GrLivArea,Int_F_Room_BsmtFullBath,Int_F_Room_BsmtHalfBath,Int_F_Room_FullBath,Int_F_Room_HalfBath,Int_F_Room_BedroomAbvGr,Int_F_Room_KitchenAbvGr,Int_F_Room_Qual_KitchenQual,Int_F_Room_TotRmsAbvGrd,Int_C_Cond_Functional,Int_F_Util_Fireplaces,Int_F_Util_Qual_FireplaceQu,Int_F_Gar_GarageType,Int_F_Gar_Year_GarageYrBlt,Int_C_Gar_Qual_GarageFinish,Int_C_Gar_Cond_GarageCars,Int_F_Gar_Room_GarageArea,Int_F_Gar_Sqft_GarageQual,Int_C_Gar_Cond_GarageCond,Int_C_PavedDrive,Int_F_Mat_Sqft_WoodDeckSF,Int_F_Porch_Sqft_OpenPorchSF,Int_F_Porch_EnclosedPorch,Int_F_Porch_Sqft_3SsnPorch,Int_F_Porch_Sqft_ScreenPorch,Int_F_Pool_Sqft_PoolArea,Int_F_Pool_Qual_PoolQC,Int_C_Mat_Fence,Int_F_Misc_MiscFeature,Int_F_Misc_Price_MiscVal,Int_F_Year_MoSold,Int_F_Year_YrSold,Ext_Type_F_SaleType,Ext_Type_F_SaleCondition,Ext_Type_F_Price_SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500


In [228]:
# Drop Column Id as it is just a numbering format and not required for our analysis
house = house.drop('Id', axis = 1)

In [229]:
print('Original column names of DF')
print(house.columns)

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [230]:
masks = ['Ext_Type', 'Ext_Type', 'Ext_F', 'Int_F', 'Ext_C', 'Ext_C', 
'Int_F', 'Int_F', 'Int_C_Util', 'Int_F', 'Ext_F',
'Ext_F', 'Ext_F', 'Ext_F', 'Int_F', 'Int_F',
'Int_F', 'Int_C', 'Ext_F_Year', 'Ext_F_Year', 'Int_C_Mat',
'Int_C_Mat', 'Int_C_Mat', 'Int_C_Mat', 'Int_C_Mat', 'Int_C_Mat_Sqft',
'Int_C_Mat_Qual', 'Int_C_Mat_Cond', 'Int_F_Mat', 'Int_F_Bsmnt_Qual', 'Int_F_Bsmnt_Cond',
'Int_C_Bsmnt', 'Int_F_Bsmnt_Qual', 'Int_F_Bsmnt_Sqft', 'Int_F_Bsmnt_Qual',
'Int_F_Bsmnt_Sqft', 'Int_F_Bsmnt_Sqft', 'Int_F_Bsmnt_Sqft', 'Int_C_Util', 'Int_C_Qual_Util',
'Int_F_Util', 'Int_F_Util', 'Int_F_Sqft', 'Int_F_Sqft', 'Int_C_Sqft',
'Int_F_Sqft', 'Int_F_Room', 'Int_F_Room', 'Int_F_Room', 'Int_F_Room',
'Int_F_Room', 'Int_F_Room', 'Int_F_Room_Qual', 'Int_F_Room',
'Int_C_Cond', 'Int_F_Util', 'Int_F_Util_Qual',  'Int_F_Gar', 'Int_F_Gar_Year',
'Int_C_Gar_Qual', 'Int_C_Gar_Cond', 'Int_F_Gar_Room', 'Int_F_Gar_Sqft', 'Int_C_Gar_Cond',
'Int_C', 'Int_F_Mat_Sqft', 'Int_F_Porch_Sqft', 'Int_F_Porch', 'Int_F_Porch_Sqft',
'Int_F_Porch_Sqft', 'Int_F_Pool_Sqft', 'Int_F_Pool_Qual', 'Int_C_Mat', 'Int_F_Misc', 'Int_F_Misc_Price',
'Int_F_Year', 'Int_F_Year', 'Ext_Type_F', 'Ext_Type_F', 'Ext_Type_F_Price']

In [231]:
# Append masks to the column names
masks = [mask+'_'+col for mask, col in zip(masks, house.columns)]
# Rename the columns
house.columns = masks
# Display new column names to cross verify the change
print('New column names of DF after prefixes were added')
print(house.columns)

Index(['Ext_Type_MSSubClass', 'Ext_Type_MSZoning', 'Ext_F_LotFrontage',
       'Int_F_LotArea', 'Ext_C_Street', 'Ext_C_Alley', 'Int_F_LotShape',
       'Int_F_LandContour', 'Int_C_Util_Utilities', 'Int_F_LotConfig',
       'Ext_F_LandSlope', 'Ext_F_Neighborhood', 'Ext_F_Condition1',
       'Ext_F_Condition2', 'Int_F_BldgType', 'Int_F_HouseStyle',
       'Int_F_OverallQual', 'Int_C_OverallCond', 'Ext_F_Year_YearBuilt',
       'Ext_F_Year_YearRemodAdd', 'Int_C_Mat_RoofStyle', 'Int_C_Mat_RoofMatl',
       'Int_C_Mat_Exterior1st', 'Int_C_Mat_Exterior2nd',
       'Int_C_Mat_MasVnrType', 'Int_C_Mat_Sqft_MasVnrArea',
       'Int_C_Mat_Qual_ExterQual', 'Int_C_Mat_Cond_ExterCond',
       'Int_F_Mat_Foundation', 'Int_F_Bsmnt_Qual_BsmtQual',
       'Int_F_Bsmnt_Cond_BsmtCond', 'Int_C_Bsmnt_BsmtExposure',
       'Int_F_Bsmnt_Qual_BsmtFinType1', 'Int_F_Bsmnt_Sqft_BsmtFinSF1',
       'Int_F_Bsmnt_Qual_BsmtFinType2', 'Int_F_Bsmnt_Sqft_BsmtFinSF2',
       'Int_F_Bsmnt_Sqft_BsmtUnfSF', 'Int_F_Bsmnt_Sqf

In [239]:
# Since we are interested only in residential properties - Drop all rows that relate to non residential
print('Original Unique values of properties:')
print(house.Ext_Type_MSZoning.unique())
# Infer from above: Only 10 properties are not residential. Have a look at them before dropping those rows
'''print(house[house['Ext_Type_MSZoning'] == 'C (all)'])'''
house = house[house['Ext_Type_MSZoning'] != 'C (all)']
print('Shape of DF after dropping non-residential properties: ')
print(house.shape)
print('New Unique values of properties: ')
print(house.Ext_Type_MSZoning.unique())

Original Unique values of properties:
['RL' 'RM' 'FV' 'RH']
Shape of DF after dropping non-residential properties: 
(1450, 80)
New Unique values of properties: 
['RL' 'RM' 'FV' 'RH']


In [136]:
# View the different unique values in each column
att_nunique_cnt = []
att_names = []

for col in house.columns:
    att_names.append(col)
    att_nunique_cnt.append(house[col].nunique())

# Convert values to a dataframe --> Transpose -->Sort based on count[View values that]
unique_value_pd = pd.DataFrame([att_names, att_nunique_cnt], index =['Attribute', 'Nunique']).T.sort_values('Nunique', ascending = False)
unique_value_pd

,Attribute,Nunique
3,LotArea,1073
45,GrLivArea,861
36,BsmtUnfSF,780
42,1stFlrSF,753
37,TotalBsmtSF,721
79,SalePrice,663
33,BsmtFinSF1,637
61,GarageArea,441
43,2ndFlrSF,417
25,MasVnrArea,327


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2. Determine any value of *changeable* property characteristics unexplained by the *fixed* ones.

---

Now that you have a model that estimates the price of a house based on its static characteristics, we can move forward with part 2 and 3 of the plan: what are the costs/benefits of quality, condition, and renovations?

There are two specific requirements for these estimates:
1. The estimates of effects must be in terms of dollars added or subtracted from the house value. 
2. The effects must be on the variance in price remaining from the first model.

The residuals from the first model (training and testing) represent the variance in price unexplained by the fixed characteristics. Of that variance in price remaining, how much of it can be explained by the easy-to-change aspects of the property?

---

**Your goals:**
1. Evaluate the effect in dollars of the renovate-able features. 
- How would your company use this second model and its coefficients to determine whether they should buy a property or not? Explain how the company can use the two models you have built to determine if they can make money. 
- Investigate how much of the variance in price remaining is explained by these features.
- Do you trust your model? Should it be used to evaluate which properties to buy and fix up?

In [139]:
# A:

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. What property characteristics predict an "abnormal" sale?

---

The `SaleCondition` feature indicates the circumstances of the house sale. From the data file, we can see that the possibilities are:

       Normal	Normal Sale
       Abnorml	Abnormal Sale -  trade, foreclosure, short sale
       AdjLand	Adjoining Land Purchase
       Alloca	Allocation - two linked properties with separate deeds, typically condo with a garage unit	
       Family	Sale between family members
       Partial	Home was not completed when last assessed (associated with New Homes)
       
One of the executives at your company has an "in" with higher-ups at the major regional bank. His friends at the bank have made him a proposal: if he can reliably indicate what features, if any, predict "abnormal" sales (foreclosures, short sales, etc.), then in return the bank will give him first dibs on the pre-auction purchase of those properties (at a dirt-cheap price).

He has tasked you with determining (and adequately validating) which features of a property predict this type of sale. 

---

**Your task:**
1. Determine which features predict the `Abnorml` category in the `SaleCondition` feature.
- Justify your results.

This is a challenging task that tests your ability to perform classification analysis in the face of severe class imbalance. You may find that simply running a classifier on the full dataset to predict the category ends up useless: when there is bad class imbalance classifiers often tend to simply guess the majority class.

It is up to you to determine how you will tackle this problem. I recommend doing some research to find out how others have dealt with the problem in the past. Make sure to justify your solution. Don't worry about it being "the best" solution, but be rigorous.

Be sure to indicate which features are predictive (if any) and whether they are positive or negative predictors of abnormal sales.

In [140]:
# A: